In [190]:
import pyproj
from pyproj import CRS, Transformer
import pandas as pd
from math import pi,sqrt,pow,atan,sin,cos,radians
import numpy as np

In [191]:
# 1) ข้อมูล ECEF ที่โจทย์ให้
data = {
    "name": ["AKSN", "AMKO", "APKN"],
    "x": [-1482251.762, -883143.728, -1037944.776],
    "y": [5925272.273, 6010905.970, 6165673.259],
    "z": [1831475.943, 1937627.074, 1255828.562]
}
df = pd.DataFrame(data)


# 2) สร้าง CRS ที่ต้องใช้
crs_ecef = CRS.from_epsg(4978)       # ECEF WGS84
crs_geod = CRS.from_epsg(4979)       # Geodetic WGS84 (lat, lon, h)
crs_utm47 = CRS.from_epsg(32647)     # UTM WGS84 Zone 47N
crs_utm48 = CRS.from_epsg(32648)     # UTM WGS84 Zone 48N


# 3) สร้าง Transformer
ecef_to_geod = Transformer.from_crs(crs_ecef, crs_geod, always_xy=True)
geod_to_utm47 = Transformer.from_crs(crs_geod, crs_utm47, always_xy=True)
geod_to_utm48 = Transformer.from_crs(crs_geod, crs_utm48, always_xy=True)

# 4) แปลงค่าพิกัด
lat, lon, h = [], [], []
utm47_e, utm47_n = [], []
utm48_e, utm48_n = [], []

for i, row in df.iterrows():
    # ECEF -> Geodetic
    lon_, lat_, h_ = ecef_to_geod.transform(row['x'], row['y'], row['z'])
    lat.append(lat_)
    lon.append(lon_)
    h.append(h_)
    
    # Geodetic -> UTM47
    e47, n47, _ = geod_to_utm47.transform(lon_, lat_, h_)
    utm47_e.append(e47)
    utm47_n.append(n47)

    # Geodetic -> UTM48
    e48, n48, _ = geod_to_utm48.transform(lon_, lat_, h_)
    utm48_e.append(e48)
    utm48_n.append(n48)

df["lat"] = lat
df["lon"] = lon
df["h_ellipsoid"] = h
df["UTM47_E"] = utm47_e
df["UTM47_N"] = utm47_n
df["UTM48_E"] = utm48_e
df["UTM48_N"] = utm48_n


# 5) โหลด Geoid TGM2017
geoid_path = r"C:\Users\user\Desktop\work\Internship\TGM2017\tgm2017.gtx"

crs_geoid = CRS.from_proj4(
    f"+proj=latlong +datum=WGS84 +geoidgrids={geoid_path}"
)
geod_to_geoid = Transformer.from_crs(crs_geod, crs_geoid, always_xy=True)


# 6) คำนวณ Orthometric Height (MSL)
H = []
for i, row in df.iterrows():
    lon_, lat_, h_ = row["lon"], row["lat"], row["h_ellipsoid"]
    # คืนค่าเป็น Orthometric height (h - N)
    _, _, H_ = geod_to_geoid.transform(lon_, lat_, h_)
    H.append(H_)

df["H_MSL"] = H

# 7) แสดงผล
print(df)


   name            x            y            z        lat         lon  \
0  AKSN -1482251.762  5925272.273  1831475.943  16.797833  104.044741   
1  AMKO  -883143.728  6010905.970  1937627.074  17.800639   98.358300   
2  APKN -1037944.776  6165673.259  1255828.562  11.431535   99.555719   

   h_ellipsoid       UTM47_E       UTM47_N        UTM48_E       UTM48_N  \
0   172.465006  1.038134e+06  1.864050e+06  398206.758456  1.857436e+06   
1   783.890729  4.319909e+05  1.968246e+06 -205186.877291  1.980664e+06   
2    -3.231108  5.606198e+05  1.263753e+06  -94702.920756  1.269308e+06   

        H_MSL  
0  197.859667  
1  822.086212  
2   25.476382  


2.พัฒนา function 4 ตัว
- UTM2LLA
- UTM2XYZ
- LLA2XYZ
- XYZ2UTM
ในงานนี้ได้ใช้พื้นฐานมาจากไฟล์ Learning_Map_Projection.ipyb ที่ได้จากการเรียน

In [192]:
#ฟังก์ชันจากการเรียน
#แปลงค่าพิกัด xyz เป็น ละติจูด ลองจิจูด และ ความสูงเหนือทรงรี  หน่วยดีกรี
def xyz2lla(x,y,z):
    ecef = '+proj=geocent +ellps=WGS84' # Cartisian   
    lla = "epsg:4326" # WGS84 Geodetic    
    transproj = pyproj.Transformer.from_crs(ecef,lla)
    lat,lon,ell_h = transproj.transform(x,y,z,radians=False)
    return lat,lon,ell_h

#แปลงค่าพิกัด ละติจูด ลองจิจูด และ ความสูงเหนือทรงรี   เป็นพิกัด UTM Zone 47N
def lla2utm(lat,lon,ell_h,zone):
    '''
        I = lat ,long unit Degree
        I = ell_h unit meters
        O = E,N,h unit meters
    '''
    lla = "epsg:4326" # WGS84 Geodetic
    utm = "epsg:326" + str(zone) # WGS84 UTM Zone 47N  
    transproj = pyproj.Transformer.from_crs(lla,utm,always_xy=True) # always_xy หมายถึงให้เรียง lon,lat
    E,N,h = transproj.transform(lon,lat,ell_h,radians=False)
    return E,N,h

In [193]:
def UTM2LLA(E, N, zone):
    '''
    Converts UTM coordinates to Latitude, Longitude, and Ellipsoidal Height.
    Input = E, N (meters), zone (UTM)
    Output = lat, lon, ell_h (degrees)
    '''
    utm_proj = "epsg:326" + str(zone)  # WGS84 UTM Zone
    lla_proj = "epsg:4326"            # WGS84 Geodetic
    
    transproj = pyproj.Transformer.from_crs(utm_proj, lla_proj, always_xy=True)
    lon, lat, ellipse_h = transproj.transform(E, N, 0) # Assuming height is 0 for 2D transformation

    return lat, lon, ellipse_h

In [194]:
def UTM2XYZ(E, N, zone):
    '''
    Converts UTM coordinates to ECEF (x, y, z).
    I = E, N (meters), zone (UTM)
    O = x, y, z (meters)
    '''
    # Step 1: Convert UTM to LLA
    lat, lon, ell_h = UTM2LLA(E, N, zone)
    
    # Step 2: Convert LLA to XYZ (using principles similar to xyz2lla but in reverse)
    ecef_proj = '+proj=geocent +ellps=WGS84'
    lla_proj = "epsg:4326"
    
    transproj = pyproj.Transformer.from_crs(lla_proj, ecef_proj)
    x, y, z = transproj.transform(lat, lon, ell_h)
    
    return x, y, z

In [195]:
def LLA2XYZ(lat, lon, ell_h):
    '''
    Converts Latitude, Longitude, and Ellipsoidal Height to ECEF (x, y, z).
    I = lat, lon (degrees), ell_h (meters)
    O = x, y, z (meters)
    '''
    ecef_proj = '+proj=geocent +ellps=WGS84' # Cartesian
    lla_proj = "epsg:4326"                 # WGS84 Geodetic
    
    transproj = pyproj.Transformer.from_crs(lla_proj, ecef_proj)
    x, y, z = transproj.transform(lat, lon, ell_h)
    
    return x, y, z

In [196]:
def XYZ2UTM(x, y, z, zone):
    '''
    Converts ECEF (x, y, z) to UTM coordinates.
    I = x, y, z (meters), zone (UTM)
    O = E, N, h (meters)
    '''
    # Step 1: Convert XYZ to LLA (using the existing function)
    ecef_proj = '+proj=geocent +ellps=WGS84'
    lla_proj = "epsg:4326"
    
    transproj = pyproj.Transformer.from_crs(ecef_proj, lla_proj)
    lat, lon, ell_h = transproj.transform(x, y, z)
    
    # Step 2: Convert LLA to UTM (using the existing function)
    utm_proj = "epsg:326" + str(zone)
    lla_proj = "epsg:4326"
    
    transproj = pyproj.Transformer.from_crs(lla_proj, utm_proj, always_xy=True)
    E, N, h = transproj.transform(lon, lat, ell_h)
    
    return E, N, h

3.แปลงพิกัดจากไฟล์ coordinate_lab1.csv จาก ECEF ไปสู่ Geodetic และ UTM Zone 47N

In [197]:
# Load the CSV file
csv_path = r"c:\\Users\\user\\Desktop\\work\\year 4\\term1\\Geodetic sur\\VScode\\Lerning_Map_Prection-513652-17240536460415\\coordinate_lab1.csv"
data = pd.read_csv(csv_path)

# Transform ECEF to Geodetic (lat, lon, h) and UTM Zone 47N
latitudes, longitudes, heights = [], [], []
utm_eastings, utm_northings, orthometric_heights = [], [], []

# Load Geoid Model
geoid_path = r"C:\\Users\\user\\Desktop\\work\\Internship\\TGM2017\\tgm2017.gtx"
crs_geoid = CRS.from_proj4(f"+proj=latlong +datum=WGS84 +geoidgrids={geoid_path}")
geod_to_geoid = Transformer.from_crs(CRS.from_epsg(4979), crs_geoid, always_xy=True)

for i, row in data.iterrows():
    try:
        # ECEF to Geodetic
        lat, lon, h = xyz2lla(row['x'], row['y'], row['z'])
        latitudes.append(lat)
        longitudes.append(lon)
        heights.append(h)

        # Geodetic to UTM Zone 47N
        e, n, _ = lla2utm(lat, lon, h, 47)
        utm_eastings.append(e)
        utm_northings.append(n)

        # Calculate Orthometric Height
        _, _, orthometric_h = geod_to_geoid.transform(lon, lat, h)
        orthometric_heights.append(orthometric_h)
    except Exception as e:
        print(f"Error processing row {i}: {e}")
        latitudes.append(None)
        longitudes.append(None)
        heights.append(None)
        utm_eastings.append(None)
        utm_northings.append(None)
        orthometric_heights.append(None)

# Add the transformed coordinates to the DataFrame
data['lat'] = latitudes
data['lon'] = longitudes
data['h'] = heights
data['UTM47_E'] = utm_eastings
data['UTM47_N'] = utm_northings
data['Orthometric_Height'] = orthometric_heights

# Display the DataFrame
print(data.head(5))

    sta             x            y             z        lat         lon  \
0  AMKO -8.831437e+05  6010905.970  1.937627e+06  17.800639   98.358300   
1  AUPG -9.448123e+05  6059317.812  1.748566e+06  16.016167   98.862603   
2  AWLK -9.568981e+05  6237552.826  9.230742e+05   8.377234   98.721705   
3  AYYA -1.134356e+06  6075002.177  1.572057e+06  14.364311  100.576764   
4  BDNG -1.394062e+06  5916171.632  1.926911e+06  17.700826  103.259067   

            h        UTM47_E       UTM47_N  Orthometric_Height  
0  783.890771  431990.890433  1.968246e+06          822.086254  
1  427.829051  485300.812239  1.770729e+06          463.927128  
2   -8.143341  469360.861589  9.260129e+05           15.891711  
3  -15.464410  670023.200099  1.588598e+06           15.550809  
4  146.757909  951971.784511  1.962201e+06          176.408836  


4.ออกแบบ Transverse Mercator ที่เหมาะกับจุฬาลงกรณ์มหาวิทยาลัย

In [198]:
# ออกแบบการฉายแผนที่แบบ Transverse mecator ที่เหมาะสมกับพื้นที่บริเวณจุฬาลงกรณ์มหาวิทยาลัย
# ในสคริปต์นี้ใช้ค่าพิกัด lat=13.736717, lon=100.533409 ซึ่งจะอยู่บริเวณตึก 3 คณะวิศวกรรมศาสตร์

# พารามิเตอร์สำหรับ TM projection
central_lat =13.738614793878359
central_lon = 100.53053933501013
scale_factor = 0.9996  # Scale factor ที่ central meridian
false_easting = 500000  # False easting (meters)
false_northing = 0  # False northing (meters)

# สร้าง projection
TM_proj_CU = CRS.from_proj4(
    f"+proj=tmerc +lat_0={central_lat} +lon_0={central_lon} +k={scale_factor} "
    f"+x_0={false_easting} +y_0={false_northing} +datum=WGS84"
)

# คำอธิบาย
# +proj=tmerc: แจ้งว่าเป็น Transverse mercator
# Central meridian เลือกเป็นบริเวณลานพระบรมรูป2รัชกาล
# ซึ่งอยู่ที่ละติจูด 13.738614793878359 และลองจิจูด 100.53053933501013 เพราะว่าอยู่ใจกลางมหาวิทยาลัย
# scale factor ที่ central meridian = 0.9996
# false_easting = 500000  > เพราะว่าเป็น Northern hemisphere
# false_northing = 0 > เพราะว่าเป็น Northern hemisphere
#  datum เลือก ellipsoid เป็น WGS84

print(TM_proj_CU)

+proj=tmerc +lat_0=13.738614793878359 +lon_0=100.53053933501013 +k=0.9996 +x_0=500000 +y_0=0 +datum=WGS84 +type=crs


5. คำนวณระยะทางบน จุด A-B

In [199]:
#คำนวณรัศมีโลก ณ R = Geocentric Latitude
def radius (B):
    # input B is Latitude in Degree unit
    # output radius in Km unit
    B = radians(B) #converting into radians
    a = 6378.137  #Radius at sea level at equator
    b = 6356.752  #Radius at poles
    c = (a**2*cos(B))**2
    d = (b**2*sin(B))**2
    e = (a*cos(B))**2
    f = (b*sin(B))**2
    R = sqrt((c+d)/(e+f))
    return R

#คำนวนสเกลแฟกเตอร์ในระบบพิกัด UTM โดยระบบพิกัดละติจูด ลองจิจูด และ โซน
def utm_scale_factor(lat,lon,zone):
    '''
        I = lat ,long unit Degree
        I = zone (UTM)
        O = scaleX, scaleY, scaleArea, mer_conv
    '''
    prj = pyproj.Proj('epsg:326' + zone)
    fc = prj.get_factors(lon,lat,radians=False)
    scaleX = fc.parallel_scale
    scaleY = fc.meridional_scale
    scaleArea = fc.areal_scale 
    mer_conv = fc.meridian_convergence
    return scaleX, scaleY, scaleArea, mer_conv

In [200]:
# Step 1: Convert ECEF to lat lon h and UTM zone 47N
A_lat, A_lon, A_h = xyz2lla(-1482251.762, 5926272.273, 1831475.943)
B_lat, B_lon, B_h = xyz2lla(-1037944.776, 6165673.259, 1255828.562)

A_e47, A_n47, _ = lla2utm(A_lat, A_lon, A_h, 47)
B_e47, B_n47, _ = lla2utm(B_lat, B_lon, B_h, 47)

# Load Geoid Model
geoid_path = r"C:\\Users\\user\\Desktop\\work\\Internship\\TGM2017\\tgm2017.gtx"
crs_geoid = CRS.from_proj4(f"+proj=latlong +datum=WGS84 +geoidgrids={geoid_path}")
geod_to_geoid = Transformer.from_crs(CRS.from_epsg(4979), crs_geoid, always_xy=True)

# Calculate Orthometric Height
_, _, A_orthometric_h = geod_to_geoid.transform(A_lon, A_lat, A_h)
_, _, B_orthometric_h = geod_to_geoid.transform(B_lon, B_lat, B_h)

# Step 2: Calculate distance on UTM zone 47N
grid_distance_47N = ((B_e47 - A_e47)**2 + (B_n47 - A_n47)**2)**0.5

# Step 3: Calculate grid scale factor (gsf) using pyproj
prj47 = pyproj.Proj("epsg:32647")
A_factors47 = prj47.get_factors(A_lon, A_lat, radians=False)
B_factors47 = prj47.get_factors(B_lon, B_lat, radians=False)

A_gsf47 = A_factors47.parallel_scale
B_gsf47 = B_factors47.parallel_scale

# Step 4: Calculate ellipsoid distance
ellipsoid_distance_47N = grid_distance_47N * (A_gsf47 + B_gsf47) / 2

# Step 5: Calculate Elevation scale factor (esf)
R_A = radius(A_lat)
R_B = radius(B_lat)
R_avg = (R_A + R_B) / 2

ESF_A = R_avg / (R_avg + A_h)
ESF_B = R_avg / (R_avg + B_h)

# Step 6: Calculate real-world distance
real_distance_47N = ellipsoid_distance_47N * (ESF_A + ESF_B) / 2

# Display results
results = pd.DataFrame({
    "Point": ["A", "B"],
    "Latitude": [A_lat, B_lat],
    "Longitude": [A_lon, B_lon],
    "Ellipsoidal Height (m)": [A_h, B_h],
    "Orthometric Height (m)": [A_orthometric_h, B_orthometric_h],
    "UTM Easting (m)": [A_e47, B_e47],
    "UTM Northing (m)": [A_n47, B_n47]
})

print("Point Information:")
print(results)

print("\nDistances:")
print(f"Grid Distance (m): {grid_distance_47N}")
print(f"Ellipsoid Distance (m): {ellipsoid_distance_47N}")
print(f"Real-World Distance (m): {real_distance_47N}")

Point Information:
  Point   Latitude   Longitude  Ellipsoidal Height (m)  \
0     A  16.795300  104.042464             1101.188307   
1     B  11.431535   99.555719               -3.231108   

   Orthometric Height (m)  UTM Easting (m)  UTM Northing (m)  
0             1126.591355     1.037898e+06      1.863762e+06  
1               25.476382     5.606198e+05      1.263753e+06  

Distances:
Grid Distance (m): 766684.6661817189
Ellipsoid Distance (m): 767767.6800195033
Real-World Distance (m): 711432.8561459411


In [201]:
# Step 1: Convert ECEF to lat lon h and UTM zone 48N
A_lat, A_lon, A_h = xyz2lla(-1482251.762, 5926272.273, 1831475.943)
B_lat, B_lon, B_h = xyz2lla(-1037944.776, 6165673.259, 1255828.562)

A_e48, A_n48, _ = lla2utm(A_lat, A_lon, A_h, 48)
B_e48, B_n48, _ = lla2utm(B_lat, B_lon, B_h, 48)

# Load Geoid Model
geoid_path = r"C:\\Users\\user\\Desktop\\work\\Internship\\TGM2017\\tgm2017.gtx"
crs_geoid = CRS.from_proj4(f"+proj=latlong +datum=WGS84 +geoidgrids={geoid_path}")
geod_to_geoid = Transformer.from_crs(CRS.from_epsg(4979), crs_geoid, always_xy=True)

# Calculate Orthometric Height
_, _, A_orthometric_h = geod_to_geoid.transform(A_lon, A_lat, A_h)
_, _, B_orthometric_h = geod_to_geoid.transform(B_lon, B_lat, B_h)

# Step 2: Calculate distance on UTM zone 48N
grid_distance_48N = ((B_e48 - A_e48)**2 + (B_n48 - A_n48)**2)**0.5

# Step 3: Calculate grid scale factor (gsf) using pyproj
prj48 = pyproj.Proj("epsg:32648")
A_factors48 = prj48.get_factors(A_lon, A_lat, radians=False)
B_factors48 = prj48.get_factors(B_lon, B_lat, radians=False)

A_gsf48 = A_factors48.parallel_scale
B_gsf48 = B_factors48.parallel_scale

# Step 4: Calculate ellipsoid distance
ellipsoid_distance_48N = grid_distance_48N * (A_gsf48 + B_gsf48) / 2

# Step 5: Calculate Elevation scale factor (esf)
R_A = radius(A_lat)
R_B = radius(B_lat)
R_avg = (R_A + R_B) / 2

ESF_A = R_avg / (R_avg + A_h)
ESF_B = R_avg / (R_avg + B_h)

# Step 6: Calculate real-world distance
real_distance_48N = ellipsoid_distance_48N * (ESF_A + ESF_B) / 2

# Display results
results_48N = pd.DataFrame({
    "Point": ["A", "B"],
    "Latitude": [A_lat, B_lat],
    "Longitude": [A_lon, B_lon],
    "Ellipsoidal Height (m)": [A_h, B_h],
    "Orthometric Height (m)": [A_orthometric_h, B_orthometric_h],
    "UTM Easting (m)": [A_e48, B_e48],
    "UTM Northing (m)": [A_n48, B_n48]
})

print("Point Information for Zone 48N:")
print(results_48N)

print("\nDistances for Zone 48N:")
print(f"Grid Distance (m): {grid_distance_48N}")
print(f"Ellipsoid Distance (m): {ellipsoid_distance_48N}")
print(f"Real-World Distance (m): {real_distance_48N}")

Point Information for Zone 48N:
  Point   Latitude   Longitude  Ellipsoidal Height (m)  \
0     A  16.795300  104.042464             1101.188307   
1     B  11.431535   99.555719               -3.231108   

   Orthometric Height (m)  UTM Easting (m)  UTM Northing (m)  
0             1126.591355    397962.839305      1.857157e+06  
1               25.476382    -94702.920756      1.269308e+06  

Distances for Zone 48N:
Grid Distance (m): 766998.1291879393
Ellipsoid Distance (m): 768420.0167833918
Real-World Distance (m): 712037.3278099351
